# 07-02 Walk-based Node Similarities

*June 12 2024*  

Going beyond local similarity measures that only consider the immediate neighborhood of nodes, we explore walk-based similarity scores that consider the number of walks between nodes.

In [1]:
import pathpyG as pp
import numpy as np

from matplotlib import pyplot as plt
from collections import defaultdict

import scipy as sp

import pandas as pd
import seaborn as sns 

plt.style.use('default')
sns.set_style("whitegrid")

plot_style = {
    'edge_color': 'grey'
}

We use the same toy example and the same functions as in the previous notebook:

In [21]:
n = pp.Graph.from_edge_list([('a', 'b'), ('b', 'c'), ('c', 'a'), ('d', 'e'), ('e', 'f'), ('f', 'g'), ('g', 'd'), ('d', 'f'), ('b', 'd')]).to_undirected()
pp.plot(n, node_label=[v for v in n.mapping.node_ids]);

In [22]:
def highlight_edges(styler, edge_df):
    styler.set_table_styles([{'selector': '.true', 'props': 'background-color: #e6ffe6;'},
    {'selector': '.false', 'props': 'background-color: #ffe6e6;'}], overwrite=False)
    styler.set_td_classes(edge_df)
    return styler

def get_node_similarities(n, score, **kwargs):
    sim_scores = defaultdict(defaultdict)
    edges = defaultdict(defaultdict)
    nodes = [v for v in n.nodes]
    for v in n.nodes:
        for w in n.nodes:
            sim_scores[v][w] = score(n, v, w, **kwargs)
            if (v,w) in n.edges:
                edges[v][w] = 'true '
            else:
                edges[v][w] = 'false '

    df = pd.DataFrame.from_dict(sim_scores, orient='index')
    edge_df = pd.DataFrame.from_dict(edges, orient='index')

    edge_df.columns = nodes
    edge_df.index = nodes
    df.columns = nodes
    df.index = nodes
    return df, edge_df

### Number of walks of length k

In the previous notebook, we motivated similarity scores using the inverse path length, a non-local score that uses the inverse of the topological distance between nodes. In our toy example, we found that the inverse path length from node $e$ to the nodes $g$ and $b$ is the same, even though nodes $e$ and $g$ are located in the same *community*, while nodes $e$ and $b$ are located in different communities (e.g. using the notion of communities introduced in L02). 

Intuitively, we may be interested in a similarity score that assigns higher similarities to nodes in the same communities as opposed to nodes in different communities. The issue with the inverse path length is that we only consider a single, i.e. the shortest, path between two nodes. However, for pairs of nodes that are located in the same densely connected community we expect *more* different paths (or walks) of a given length to exist, compared to pairs of nodes located in different clusters. 

A simple way to account for this is to compute the total number of walks of a given length $k$. Referring to L02, we can compute this based on the $k$-th power of the (binary) adjacency matrix. The following function calculates the number of walks of exactly length two between any given pair of nodes. 

In [67]:
def num_walks(n, v, w, **kwargs):
    A = n.get_sparse_adj_matrix().todense()
    Ak = A**kwargs['k']
    return Ak[n.mapping.to_idx(v), n.mapping.to_idx(w)]

Using this as a "similarity" score, we find that node "e" is now more similar to node "g" compared to node $b$, even though it is connected by a path of length two two both "g" and "b". Due to the dense connectivity in the community two different paths of length two exist from $e$ to $g$, while a single path of length two exists from $e$ to $b$.

In [68]:
df, edges = get_node_similarities(n, num_walks, k=2)
df.style.pipe(highlight_edges, edges)

,a,b,c,d,e,f,g
a,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
b,1.000000,3.000000,1.000000,0.000000,1.000000,1.000000,1.000000
c,1.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000
d,1.000000,0.000000,1.000000,4.000000,1.000000,2.000000,1.000000
e,0.000000,1.000000,0.000000,1.000000,2.000000,1.000000,2.000000
f,0.000000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000
g,0.000000,1.000000,0.000000,1.000000,2.000000,1.000000,2.000000


A problem with this approach is that we somehow must adjust the parameter $k$, which gives the length of the paths that we are interested in, to the size of the communities. If, for instance, we calculate the number of paths of length three, we find that two paths exist from node $f$ to $b$, because those paths can take different routes within the community of node $b$, while taking the same edge connecting this community to node $b$.

In [69]:
df, edges = get_node_similarities(n, num_walks, k=3)
df.style.pipe(highlight_edges, edges)

,a,b,c,d,e,f,g
a,2.000000,4.000000,3.000000,1.000000,1.000000,1.000000,1.000000
b,4.000000,2.000000,4.000000,6.000000,1.000000,2.000000,1.000000
c,3.000000,4.000000,2.000000,1.000000,1.000000,1.000000,1.000000
d,1.000000,6.000000,1.000000,4.000000,6.000000,6.000000,6.000000
e,1.000000,1.000000,1.000000,6.000000,2.000000,5.000000,2.000000
f,1.000000,2.000000,1.000000,6.000000,5.000000,4.000000,5.000000
g,1.000000,1.000000,1.000000,6.000000,2.000000,5.000000,2.000000


In [70]:
df, edges = get_node_similarities(n, num_walks, k=4)
df.style.pipe(highlight_edges, edges)

,a,b,c,d,e,f,g
a,7.000000,6.000000,6.000000,7.000000,2.000000,3.000000,2.000000
b,6.000000,14.000000,6.000000,6.000000,8.000000,8.000000,8.000000
c,6.000000,6.000000,7.000000,7.000000,2.000000,3.000000,2.000000
d,7.000000,6.000000,7.000000,24.000000,10.000000,16.000000,10.000000
e,2.000000,8.000000,2.000000,10.000000,11.000000,10.000000,11.000000
f,3.000000,8.000000,3.000000,16.000000,10.000000,16.000000,10.000000
g,2.000000,8.000000,2.000000,10.000000,11.000000,10.000000,11.000000


Above, we have only counted walks of a specific length. We can instead take the sum of all matrix powers to calculate the number of walks with a length up to a given value $k$.

In [71]:
def num_walks(n, v, w, **kwargs):
    A = n.get_sparse_adj_matrix().todense()
    Ak = A.copy()
    for i in range(2, kwargs['k']+1):
        Ak += (A**kwargs['k'])
    return Ak[n.mapping.to_idx(v), n.mapping.to_idx(w)]

In [72]:
df, edges = get_node_similarities(n, num_walks, k=2)
df.style.pipe(highlight_edges, edges)

,a,b,c,d,e,f,g
a,2.000000,2.000000,2.000000,1.000000,0.000000,0.000000,0.000000
b,2.000000,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000
c,2.000000,2.000000,2.000000,1.000000,0.000000,0.000000,0.000000
d,1.000000,1.000000,1.000000,4.000000,2.000000,3.000000,2.000000
e,0.000000,1.000000,0.000000,2.000000,2.000000,2.000000,2.000000
f,0.000000,1.000000,0.000000,3.000000,2.000000,3.000000,2.000000
g,0.000000,1.000000,0.000000,2.000000,2.000000,2.000000,2.000000


In [73]:
df, edges = get_node_similarities(n, num_walks, k=3)
df.style.pipe(highlight_edges, edges)

,a,b,c,d,e,f,g
a,4.000000,9.000000,7.000000,2.000000,2.000000,2.000000,2.000000
b,9.000000,4.000000,9.000000,13.000000,2.000000,4.000000,2.000000
c,7.000000,9.000000,4.000000,2.000000,2.000000,2.000000,2.000000
d,2.000000,13.000000,2.000000,8.000000,13.000000,13.000000,13.000000
e,2.000000,2.000000,2.000000,13.000000,4.000000,11.000000,4.000000
f,2.000000,4.000000,2.000000,13.000000,11.000000,8.000000,11.000000
g,2.000000,2.000000,2.000000,13.000000,4.000000,11.000000,4.000000


In [76]:
df, edges = get_node_similarities(n, num_walks, k=10)
df.style.pipe(highlight_edges, edges)

,a,b,c,d,e,f,g
a,10827.000000,16885.000000,10819.000000,26235.000000,16218.000000,20907.000000,16218.000000
b,16885.000000,30996.000000,16885.000000,43993.000000,30924.000000,37764.000000,30924.000000
c,10819.000000,16885.000000,10827.000000,26235.000000,16218.000000,20907.000000,16218.000000
d,26235.000000,43993.000000,26235.000000,78138.000000,49321.000000,64027.000000,49321.000000
e,16218.000000,30924.000000,16218.000000,49321.000000,34380.000000,42481.000000,34380.000000
f,20907.000000,37764.000000,20907.000000,64027.000000,42481.000000,54054.000000,42481.000000
g,16218.000000,30924.000000,16218.000000,49321.000000,34380.000000,42481.000000,34380.000000


## Katz index

The issue with this approach is two-fold: We first need to specify the parameter $k$ up to which we want to consider walks, which is likely to depend on the diameter of the network. Moreover, we assign equal importance to paths of any length and there are (most likely) many more paths with larger length $k$ compared to smaller length $k$.

We can address this with the so-called Katz index, which is defined as the following infinite sum:

$$ K(v,w) = \sum_{l=1}^{\infty} \beta^l |\text{walks}_{v,w}^{l}| = \beta A_{vw} + \beta^2(A^2)_{vw} + \beta^3 (A^3)_{vw}$$

where $\text{walks}_{vw}^l$ is the set of all walks of exactly length $l$ that start in $v$ and end in $w$. $\beta$ is a damping factor, where smaller values of $\lambda$ assign more weight to shorter walks.

If $\beta$ is chosen smaller than the reciprocal largest eigenvalue of $A$, this converges to the following equation (in matrix form):

$$ \mathbf{S} = \left(\mathbf{I}- \beta \mathbf{A} \right)^{-1} - \mathbf{I} $$

This implies that we can calculate a matrix $\mathbf{S}$ that captures all pairwise node similarities by a simple inversion of matrix $\mathbf{I}-\beta \mathbf{A}$ (and subtracting the identity matrix). Using `scipy` and `pathpy` we can compute this as follows:

In [83]:
def katz_index(network: pp.Graph, v, w, **kwargs) -> float:
    A = network.get_sparse_adj_matrix()
    I = sp.sparse.identity(network.N)
    S = sp.sparse.linalg.inv(I - kwargs['beta'] * A) - I
    return S[network.mapping.to_idx(v), network.mapping.to_idx(w)]

The resulting similarity score depends on a parameter $\beta$ that controls the dampening factor of longer walks. For small $\beta$ the similarity is mostly based on very short walks, i.e. node pairs that are not directly connected tend to have small similarity.

In [84]:
katz_index(n, 'e', 'g', beta=0.02)

0.0008178287973506426

By increasing $\beta$ we can increase the influence of longer walks on the similarity scores:

In [85]:
katz_index(n, 'e', 'g', beta=0.2)

0.12958435772871946

In [86]:
df, edges = get_node_similarities(n, katz_index, beta=0.02)
df.style.pipe(highlight_edges, edges)

,a,b,c,d,e,f,g
a,0.000817,0.020433,0.020425,0.000409,0.000008,0.000009,0.000008
b,0.020433,0.001218,0.020433,0.020049,0.000409,0.000417,0.000409
c,0.020425,0.020433,0.000817,0.000409,0.000008,0.000009,0.000008
d,0.000409,0.020049,0.000409,0.001636,0.020450,0.020851,0.020450
e,0.000008,0.000409,0.000008,0.020450,0.000818,0.020442,0.000818
f,0.000009,0.000417,0.000009,0.020851,0.020442,0.001235,0.020442
g,0.000008,0.000409,0.000008,0.020450,0.000818,0.020442,0.000818


In [87]:
df, edges = get_node_similarities(n, katz_index, beta=0.2)
df.style.pipe(highlight_edges, edges)

/opt/conda/lib/python3.10/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:602: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/opt/conda/lib/python3.10/site-packages/scipy/sparse/linalg/_matfuncs.py:76: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  Ainv = spsolve(A, I)


,a,b,c,d,e,f,g
a,0.115016,0.293399,0.281683,0.070293,0.018337,0.021394,0.018337
b,0.293399,0.173594,0.293399,0.281174,0.073350,0.085575,0.073350
c,0.281683,0.293399,0.115016,0.070293,0.018337,0.021394,0.018337
d,0.070293,0.281174,0.070293,0.265281,0.330073,0.385086,0.330073
e,0.018337,0.073350,0.018337,0.330073,0.129584,0.317848,0.129584
f,0.021394,0.085575,0.021394,0.385086,0.317848,0.204156,0.317848
g,0.018337,0.073350,0.018337,0.330073,0.129584,0.317848,0.129584


## Leicht-Holme-Newman Index

One issue of the Katz index is that it does not correct for the number of walks that we can expect based on the degrees of nodes. To address this, we can define an index that divides the number of walks between each pair of nodes by the number of expected walks. 

This idea yields the Leicht-Holme-Newman index, which can be rewritten to the following equation: 

$$ S = D^{-1}\left(I-\frac{\alpha A}{\lambda_1} \right)^{-1}D^{-1} $$

where $\alpha$ is a free parameter in $\left[0, 1\right]$ and $\lambda_1$ is the largest eigenvalue of the adjacency matrix. Like for the Katz index, values of $\alpha$ assign more weight to short paths. We can implement it as follows:

In [93]:
def LHN_index(network: pp.Graph, v, w, **kwargs) -> float:
    A = network.get_sparse_adj_matrix()
    ev = sp.sparse.linalg.eigs(A, which="LM", k=2, return_eigenvectors=False)
    m = network.M/2
    eigenvalues_sorted = np.sort(np.absolute(ev))
    lambda_1 = eigenvalues_sorted[1]
    D = sp.sparse.diags(pp.statistics.degree_sequence(network))
    I = sp.sparse.identity(network.N)
    S = 2*m*lambda_1*sp.sparse.linalg.inv(D) * sp.sparse.linalg.inv(I - kwargs['alpha']*A/lambda_1)*sp.sparse.linalg.inv(D)
    return S[network.mapping.to_idx(v), network.mapping.to_idx(w)]


In [94]:
LHN_index(n, 'e', 'g', alpha=0.02)

0.0013079553726262417

In [95]:
LHN_index(n, 'e', 'g', alpha=0.2)

0.14353902083713282

In [96]:
df, edges = get_node_similarities(n, LHN_index, alpha=0.02)
df.style.pipe(highlight_edges, edges)

,a,b,c,d,e,f,g
a,12.479983,0.060445,0.090663,0.000327,0.000005,0.000003,0.000005
b,0.060445,5.546951,0.060445,0.030009,0.000436,0.000293,0.000436
c,0.090663,0.060445,12.479979,0.000327,0.000005,0.000003,0.000005
d,0.000327,0.030009,0.000327,3.120322,0.045339,0.030442,0.045339
e,0.000005,0.000436,0.000005,0.045339,12.479972,0.060449,0.001308
f,0.000003,0.000293,0.000003,0.030442,0.060449,5.546954,0.060449
g,0.000005,0.000436,0.000005,0.045339,0.001308,0.060449,12.479973


In [97]:
df, edges = get_node_similarities(n, LHN_index, alpha=0.2)
df.style.pipe(highlight_edges, edges)

/opt/conda/lib/python3.10/site-packages/scipy/sparse/linalg/_matfuncs.py:76: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  Ainv = spsolve(A, I)
/opt/conda/lib/python3.10/site-packages/scipy/sparse/linalg/_matfuncs.py:76: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  Ainv = spsolve(A, I)
/opt/conda/lib/python3.10/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:602: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve


,a,b,c,d,e,f,g
a,12.620581,0.657490,0.981365,0.036191,0.005656,0.004024,0.005656
b,0.657490,5.639150,0.657490,0.310400,0.048508,0.034514,0.048508
c,0.981365,0.657490,12.620579,0.036191,0.005656,0.004024,0.005656
d,0.036191,0.310400,0.036191,3.191628,0.498773,0.354884,0.498773
e,0.005656,0.048508,0.005656,0.498773,12.622218,0.661767,0.143539
f,0.004024,0.034514,0.004024,0.354884,0.661767,5.643840,0.661767
g,0.005656,0.048508,0.005656,0.498773,0.143539,0.661767,12.622205
